In [10]:
import numpy as np
import pandas as pd
import time
from nba_api.stats.static import players
from nba_api.stats.endpoints import draftcombinestats, playercareerstats

In [8]:
all_players = players.get_players()

# We only want ACTIVE players
active_players = [p for p in all_players if p["is_active"] == True]
ids = [p["id"] for p in active_players]

len(active_players)

519

In [11]:
time.sleep(1) 

In [47]:
# Get the combine data of all active players

endpoint = draftcombinestats.DraftCombineStats(league_id = "00", season_all_time="All Time")
combine_stats = endpoint.get_data_frames()[0]
combine_stats.to_csv("./combine_stats")
combine_stats

,SEASON,PLAYER_ID,FIRST_NAME,LAST_NAME,PLAYER_NAME,POSITION,HEIGHT_WO_SHOES,HEIGHT_WO_SHOES_FT_IN,HEIGHT_W_SHOES,HEIGHT_W_SHOES_FT_IN,...,STANDING_REACH_FT_IN,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,STANDING_VERTICAL_LEAP,MAX_VERTICAL_LEAP,LANE_AGILITY_TIME,MODIFIED_LANE_AGILITY_TIME,THREE_QUARTER_SPRINT,BENCH_PRESS
0,2002,2403,Nene,,Nene,PF,81.25,6' 9.25'',NaN,None,...,9' 1'',NaN,NaN,NaN,30.0,34.0,10.73,NaN,3.19,16.0
1,2007,12204,Mohamed,Abukar,Mohamed Abukar,SF,80.00,6' 8'',81.75,6' 9.75'',...,8' 7'',8.90,NaN,NaN,30.5,35.0,11.78,NaN,3.37,15.0
2,2020,1630173,Precious,Achiuwa,Precious Achiuwa,PF,79.50,6'7.50'',80.70,6'8.75'',...,9'0.50'',6.70,9.0,10.00,NaN,NaN,NaN,NaN,NaN,NaN
3,2005,101165,Alex,Acker,Alex Acker,SG,75.75,6' 3.75'',76.75,6' 4.75'',...,8' 6.5'',NaN,NaN,NaN,28.0,32.0,11.67,NaN,3.35,11.0
4,2012,203112,Quincy,Acy,Quincy Acy,PF,78.50,6' 6.5'',79.75,6' 7.75'',...,8' 10.5'',7.10,9.5,11.00,32.0,37.0,10.48,NaN,3.28,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390,2017,1962937123,Omer,Yurtseven,Omer Yurtseven,C,82.50,6' 10.5'',83.75,6' 11.75'',...,9' 0.5'',7.60,9.0,10.50,26.0,31.5,12.85,3.30,3.45,6.0
1391,2012,203092,Tyler,Zeller,Tyler Zeller,PF-C,83.25,6' 11.25'',84.50,7' 0.5'',...,8' 8.5'',6.40,9.0,9.75,30.0,34.0,11.13,NaN,3.40,16.0
1392,2013,203469,Cody,Zeller,Cody Zeller,C-PF,82.75,6' 10.75'',84.25,7' 0.25'',...,8' 10'',4.75,8.5,10.50,35.5,37.5,10.82,2.69,3.15,17.0
1393,2003,2583,Derrick,Zimmerman,Derrick Zimmerman,PG,73.00,6' 1'',NaN,None,...,8' .25'',5.30,NaN,NaN,34.0,40.5,11.09,NaN,3.10,13.0


In [48]:
combine_stats.keys()

Index(['SEASON', 'PLAYER_ID', 'FIRST_NAME', 'LAST_NAME', 'PLAYER_NAME',
       'POSITION', 'HEIGHT_WO_SHOES', 'HEIGHT_WO_SHOES_FT_IN',
       'HEIGHT_W_SHOES', 'HEIGHT_W_SHOES_FT_IN', 'WEIGHT', 'WINGSPAN',
       'WINGSPAN_FT_IN', 'STANDING_REACH', 'STANDING_REACH_FT_IN',
       'BODY_FAT_PCT', 'HAND_LENGTH', 'HAND_WIDTH', 'STANDING_VERTICAL_LEAP',
       'MAX_VERTICAL_LEAP', 'LANE_AGILITY_TIME', 'MODIFIED_LANE_AGILITY_TIME',
       'THREE_QUARTER_SPRINT', 'BENCH_PRESS'],
      dtype='object')

In [49]:
indices = combine_stats.PLAYER_ID.isin(ids)
combine_stats = combine_stats[indices]
combine_stats = pd.DataFrame(combine_stats, columns=['PLAYER_ID', 'PLAYER_NAME',
       'POSITION', 'HEIGHT_WO_SHOES', 'WEIGHT', 'WINGSPAN',
       'STANDING_REACH',
       'STANDING_VERTICAL_LEAP',
       'MAX_VERTICAL_LEAP', 'LANE_AGILITY_TIME'])

# Remove rows with missing data
combine_stats = combine_stats.dropna()
combine_stats.PLAYER_ID = combine_stats.PLAYER_ID.astype("int")
combine_stats.to_csv("./combine_stats")
len(combine_stats)

276

In [50]:
ids = list(combine_stats.PLAYER_ID)
data = []

for idx in ids:
    time.sleep(1) 
    endpoint = playercareerstats.PlayerCareerStats(player_id = idx, per_mode36 = "Per36")
    career_stats = endpoint.career_totals_regular_season.get_data_frame()
    if int(career_stats["GP"]) >= 82:
        data.append({
            "PLAYER_ID": career_stats["PLAYER_ID"],
            "PTS": float(career_stats["PTS"]),
            "GP": int(career_stats["GP"])
        })

In [51]:
points_data = pd.DataFrame(data)
points_data.to_csv("./points_data")
len(points_data)

,PLAYER_ID,PTS,GP
0,"0 2403 Name: PLAYER_ID, dtype: int64",15.6,965
1,"0 203500 Name: PLAYER_ID, dtype: int64",12.8,588
2,"0 1628389 Name: PLAYER_ID, dtype: int64",16.3,287
3,"0 200746 Name: PLAYER_ID, dtype: int64",20.4,1029
4,"0 1628960 Name: PLAYER_ID, dtype: int64",16.1,126
...,...,...,...
216,"0 1629026 Name: PLAYER_ID, dtype: int64",10.2,151
217,"0 1629684 Name: PLAYER_ID, dtype: int64",8.8,132
218,"0 1626153 Name: PLAYER_ID, dtype: int64",13.2,334
219,"0 201152 Name: PLAYER_ID, dtype: int64",15.7,1033


In [52]:
combine_stats.PLAYER_ID = combine_stats.PLAYER_ID.astype("int")
points_data.PLAYER_ID = points_data.PLAYER_ID.astype("int")
all_data = points_data.merge(combine_stats, how="inner", on="PLAYER_ID")
all_data = all_data.dropna()
all_data.to_csv("./all_data")

all_data

,PLAYER_ID,PTS,GP,PLAYER_NAME,POSITION,HEIGHT_WO_SHOES,WEIGHT,WINGSPAN,STANDING_REACH,STANDING_VERTICAL_LEAP,MAX_VERTICAL_LEAP,LANE_AGILITY_TIME
0,2403,15.6,965,Nene,PF,81.25,253,88.50,109.0,30.0,34.0,10.73
1,203500,12.8,588,Steven Adams,C,82.75,254.5,88.50,109.5,28.5,33.0,11.85
2,1628389,16.3,287,Edrice Adebayo,PF-C,80.75,242.6,86.75,108.0,33.5,38.5,11.94
3,200746,20.4,1029,LaMarcus Aldridge,PF-C,82.00,234,88.75,110.0,26.5,34.0,12.02
4,1628960,16.1,126,Grayson Allen,SG,75.00,198,79.25,97.0,32.5,40.5,10.31
...,...,...,...,...,...,...,...,...,...,...,...,...
228,1629026,10.2,151,Kenrich Williams,SF,78.00,200.4,79.25,103.5,28.0,33.0,10.86
229,1629684,8.8,132,Grant Williams,PF,77.75,240.2,81.75,104.5,26.0,31.5,10.83
230,1626153,13.2,334,Delon Wright,PG,76.25,181.4,79.50,101.5,29.5,31.0,11.16
231,201152,15.7,1033,Thaddeus Young,SF,77.75,210,83.50,106.0,34.5,37.0,11.06
